In [1]:
## reward is just a number which makes sense relative to each other
# s, a -->> r, s'

### EPISODES
# episode - represents one run of the game
# RL agent will learn across many episodes
# num of episodes is a hyperparameter..
## PLAYING TIC TAC TOE IS AN EPISODIC TASK because you can play it again and again.
## End of an episode -- Terminal State
# Each episode is a fresh start

## NOTES ON ASSIGNING REWARDS
# We prgrammers are the coaches to an AI
## Reward is something we give to the agent

## Value Functions = expected value of your final reward.
## we dont just thing about immediate rewards.
## Delayed Rewards
# Note in Credit Assignment Problem we asked what action we took in past that is leading to the rewards we are getting now,
## In Delayed rewards -- how is the action I am doing now related to the potential reward I may receive in future..-- PLANNING

## VALUE TELLS THE FUTURE GOODNESS OF A STATE
# V(s) THE VALUE OF A STATE is a measure of future rewards we may receive being in that state.
## Rewards are immediate.
# WE CHOOSE ACTIONS BASED ON VALUES OF A STATE, NOT ON THE REWARD WE WILL GET BY GOING TO THAT STATE,

## value is a fast and efficient way of searching a game tree.
## 19683 possible states.
## O(1) constant time lookup because a state directly looks up to a value.
# V(s) = E[all future rewards|S(t) = s]
# In TTT V(s) acn be interpreted as probability of winning after arriving in state s
# After initialization of the value func. we have to update it, V(s) <- V(s) + alpha*(V(s') - V(s))
# Note value of terminal state will never be updated,,
## Since value function is not accurate we will use epsilon greedy strategy to make a tradeoff bw exploration and exploitaton..
## For any particular state we are only going to update the values for the states that were in that episode.
##we are moving V(s) closer to V(s')


In [2]:
## Implementation ###
import numpy as np
length = 3

## we will have to use OOP approach,
## Two main objects the agent and the environment..
## There will be two instances of agent interacting with a single instance of environment..
def play_game(p1, p2, env, draw = False):
	## Loop untill the game is over..
	current_player = None
	while not env.game_over():
		## switch/alternate bw the players
		if current_player == p1: ## player1 will go first
			current_player = p2
		else:
			current_player = p1

		## Displaying the board func. to display what positions are currently occupied..
		if draw:
			if draw == 1 and current_player == p1: ## board only gets drawn once.
				env.draw_board()
			if draw == 2 and current_player == p2:
				env.draw_board()

		# we need current player to perform an action which updates the environment and hence the state
		current_player.take_action(env)	

		## update state histories of all the agents
		state = env.get_state()
		p1.update_state_history(state)
		p2.update_state_history(state)
	
	#we need an update function that updates the agents internal estimate of the value fn
	# This is where the update eqn will go
	## The update fn has to accept the env as well since it has to query the most current rewards from the env.
	if draw:
		env.draw_board() ## draw the board one last time to see who won.
	
    ## Doing the value fn update, since the episode is now over.
	p1.update(env)
	p2.update(env)

In [3]:
## Initializing the value fn V(s) for all states
# 1 = for any state where the player wins
# 0 = for any state where the player looses or its a draw
# 0.5 otherwise
def initialize_Vx(env, state_winner_triple):
	V = np.zeros(env.num_states)
	for state, winner, ended in state_winner_triple:
		if ended:
			if winner == env.x:
				v = 1
			else:
				v = 0
		else:
			v = 0.5
		V[state] = v
	return V

def initialize_Vo(env, state_winner_triple):
	V = np.zeros(env.num_states)
	for state, winner, ended in state_winner_triple:
		if ended:
			if winner == env.o:
				v = 1
			else:
				v = 0
		else:
			v = 0.5
		V[state] = v
	return V

In [4]:
## we need to ennumerate every possible state and assign the values
# How to enumerate??
## With Game Tree the problem is that we will same tree same tree more than once in the tree
## Game Tree will lead to 9 factorial possible states which is much grater than 3^9

## So we will use permutations of length N(Enumerating states recursively)
## Function : get_state_hash_and_winner
# returns a list of triples(state, winner, board)
## state = configuration of the board as a hashed integer
## winner is None if ended is false
# ips: env, i, j i,j lie bw 0 and 2

def get_state_hash_and_winner(env, i = 0, j = 0):
	results = []
	for v in (0, env.x, env.o):
		env.board[i][j] = v
		if j == 2:
			if i == 2:
				## if i =2 and j=2 then we are filling last location in the board,
				## Which means now we should check for who the winner is.. and whether or not the game is ended..
				## if the board is full, collect the results and return
				## note that the board can even be filled with all 0's but that dosent mean game over. 
				# it just means that my permutation is over.. 
				state = env.get_state()
				ended = env.game_over(forced_recalc = True)
				winner = env.winner
				results.append((state, winner, ended))
			else:
 				results += get_state_hash_and_winner(env, i+1, 0)
		else:
			results += get_state_hash_and_winner(env, i, j+1)	
	return results

In [5]:
## Environment Class
# Thing which agent interacts with
class Env:
    def __init__(self):
        self.board = np.zeros((length, length))
        self.x = -1 # Represents an x on board
        self.o = 1 # Represents an o on board
        self.winner = None
        self.ended =  False ## Becomes True at the end of the game
        self.num_states = 3**(length*length)
        
    def is_empty(self, i, j): ## Useful fn for agent because the agent needs to check if its valid position or not   
        return self.board[i][j] == 0
    
    def reward(self, sym): # 1 if we win the game; ow = 0
        ## a reward is given as feedback on every state transition
        if not self.game_over():
            return 0
        ## sym is the symbol of the agent
        # sym will be either self.x or self.o
        return 1 if self.winner == sym else 0
    
    ## Representing states
    ## Map each state to a number, so we can store value fn as a numpy array.
    ## 19683 possible states' not too big
    ## 3 possible symbols in each location we will covert ternay to decimal no's
    def get_state(self):
        h = 0
        k = 0
        for i in range(length):
            for j in range(length):
                if self.board[i][j] == 0:
                    v = 0
                elif self.board[i][j] == self.x:
                    v = 1
                elif self.board[i][j] == self.o:
                    v = 2
                h += (3**k)*v		## Hashed integer
                k += 1
        return h
    
    def game_over(self, forced_recalc = False):
        if not forced_recalc and self.ended:
            return self.ended
        ## Check for winner
        ## row check
        for i in range(length):
            for player in (self.x, self.o):
                if self.board[i].sum() == player*length:
                    self.winner = player
                    self.ended = True
                    return True
                
        ## column check
        for j in range(length):
            for player in (self.x, self.o):
                if self.board[:,j].sum() == player*length:
                    self.winner = player
                    self.ended = True
                    return True
                
        ## Diagonal Check
        # a. top left to bottom right
        for player in (self.x, self.o):
            if self.board.trace() == player*length:
                self.winner = player
                self.ended = True
                return True
                
        # b. top right to bottom left
            if np.fliplr(self.board).trace() == player*length:
                self.winner = player
                self.ended = True
                return True
            
        ## Draw Check
        if np.all((self.board == 0) ==  False):
            # winner stays None
            self.winner = None
            self.ended = True
            return True
        
        ## Game is not over
        self.winner = None
        return False
    
    def draw_board(self):
        for i in range(length):
            print('------------------')
            for j in range(length):
                print(' ', end="")
                if self.board[i][j] == self.x:
                    print('x', end="")
                elif self.board[i][j] == self.o:
                    print('o', end="")
                else:
                    print(' ', end="")
            print('')        
        print('------------------')     

In [6]:
## The Agent Class-- everything that contains the AI
# It has to interact with the environment
class Agent:
    def __init__(self, eps=0.1, alpha = 0.5):
        self.eps = eps # probability of choosing random action instead of greedy
        self.alpha = alpha ## The learning rate
        self.verbose = False
        self.state_history = []
        
    def setV(self, V):    
        self.V = V
        
    def set_symbol(self, sym):
        self.sym = sym
        
    def set_verbose(self, v):    
        self.verbose = v
        
    def reset_history(self):## At the end of episode.
        self.state_history = []
        
    ## with probability eps we will just do random action
    def take_action(self, env):
        # Choose an action based on epsilon greedy strategy
        r = np.random.rand()
        best_state = None
        if r < self.eps:
            if self.verbose:
                print('Taking a random action')
            possible_moves = []    
            for i in range(length):
                for j in range(length):
                    if env.is_empty(i, j):
                        possible_moves.append((i,j))
            idx = np.random.choice(len(possible_moves)) # uniform probability
            next_move = possible_moves[idx]            
        else: ## Greedy part of epsilon greedy
            pos2value = {}
            next_move = None
            best_value = -1
            for i in range(length):
                for j in range(length):
                    if env.is_empty(i, j):            
                        ## what is the state if we made this move
                        env.board[i][j] = self.sym
                        state = env.get_state()
                        env.board[i][j] = 0 # we are just checking
                        pos2value[(i,j)] = self.V[state]
                        if self.V[state] > best_value:
                            best_value = self.V[state]
                            best_state = state
                            next_move = (i,j)
                            
            # If verbose draw the board with values.                
            if self.verbose:
                print('Taking a greedy action')
                ## PRINT THE BOARD WITH VALUES
                for i in range(length):
                    print('------------------')
                    for j in range(length):
                        if env.is_empty(i,j):
                            #print the value
                            print(" %.2f|" % pos2value[(i,j)], end="")
                        else:
                            print("  ", end="")
                            if env.board[i,j] == env.x:
                                print("x  |", end="")
                            elif env.board[i,j] == env.o:
                                print("o  |", end="")
                            else:
                                print("   |", end="")
                    print("")
                print("------------------")

        # make the move
        env.board[next_move[0], next_move[1]] = self.sym    
                                
    def update_state_history(self,s):
        self.state_history.append(s)
        
    def update(self, env):    
        ## Backtracking over all the states.
        # we will only do this at the end of an episode.
        # V(next_state) = reward, if its the most current state.
        # update for the value of a state depends on the value of next state.
        reward = env.reward(self.sym)
        target = reward
        for prev in reversed(self.state_history):
            value = self.V[prev] + self.alpha*(target - self.V[prev])
            self.V[prev] = value
            target = value
        self.reset_history()    

In [7]:
class Human:
    def __init__(self):
        pass
    
    def set_symbol(self, sym):
        self.sym = sym
        
    def take_action(self, env):    
        while True:
            ## break if we make a legal move
            move = input('enter coordinate (i,j) for your next move')
            i,j = move.split(',')
            i = int(i)
            j = int(j)
            if env.is_empty(i,j):
                env.board[i][j] = self.sym
                break
        
    def update_state_history(self,s):
        pass     
    
    def update(self, env):
        pass

In [8]:
## Main fn
if __name__ == '__main__':
    # first step will be to create two AI's
    p1 = Agent()
    p2 = Agent()
    
    # Initiaizing the AI
    # set initial V for p1 and p2
    env = Env()
    state_winner_triple = get_state_hash_and_winner(env)
    
    ## Initializing their value fns
    Vx = initialize_Vx(env, state_winner_triple)
    p1.setV(Vx)
    Vo = initialize_Vo(env, state_winner_triple)
    p2.setV(Vo)
    
    ## Give each player their symbol
    p1.set_symbol(env.x)
    p2.set_symbol(env.o)
    
## Training of AI    
## Now we play 10k games of AI vs AI

T = 10000
for t in range(T):
    if t%200 == 0:
        print(t)
    play_game(p1, p2, Env())    

0
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200
6400
6600
6800
7000
7200
7400
7600
7800
8000
8200
8400
8600
8800
9000
9200
9400
9600
9800


In [17]:
## Play Human vs Agent
human = Human()
human.set_symbol(env.o)
while True:
    p1.set_verbose(True)
    play_game(p1, human, Env(), draw = 2)
    answer = input('play_again? [Y/n]')
    if answer and answer.lower()[0] == 'n':
        break

Taking a greedy action
------------------
 0.61| 0.49| 0.52|
------------------
 0.61| 0.66| 0.52|
------------------
 0.60| 0.65| 0.57|
------------------
------------------
      
------------------
   x  
------------------
      
------------------
enter coordinate (i,j) for your next move2,2
Taking a greedy action
------------------
 0.62| 0.20| 0.65|
------------------
 0.68|  x  | 0.59|
------------------
 0.64| 0.65|  o  |
------------------
------------------
      
------------------
 x x  
------------------
     o
------------------
enter coordinate (i,j) for your next move1,2
Taking a greedy action
------------------
 0.05| 0.12| 0.63|
------------------
  x  |  x  |  o  |
------------------
 0.25| 0.25|  o  |
------------------
------------------
     x
------------------
 x x o
------------------
     o
------------------
enter coordinate (i,j) for your next move2,0
Taking a greedy action
------------------
 0.00| 0.12|  x  |
------------------
  x  |  x  |  o  |
-------